In [1]:
# --- 1. Cài đặt các thư viện cần thiết ---
!pip install streamlit plotly pandas seaborn pyngrok -q
!pip install pyngrok --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 961.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00


In [19]:
# --- 2. Tạo file app.py ---
# Nội dung file: app.py
app_code = '''

import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
import numpy as np
import os # Added for path operations
import base64 # Added for image encoding

# ====== CẤU HÌNH GIAO DIỆN TỐI ======
st.set_page_config(
    page_title="AirVision Analytics",
    page_icon="🌍",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Tùy chỉnh CSS cho giao diện tối
def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

# Tạo file CSS tạm với theme tối
custom_css = """
:root {
    --primary-color: #4a8fe7;
    --secondary-color: #2c3e50;
    --accent-color: #ff6b6b;
    --dark-color: #1a1a1a;
    --light-color: #f8f9fa;
}


.stApp {
    background-color: var(--dark-color);
    color: #ffffff;
}

/* Header */
header {
    background: linear-gradient(135deg, var(--primary-color), #3a7bd5);
    color: white !important;
    padding: 1rem 2rem !important;
    border-radius: 0 0 10px 10px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.3);
}

/* Sidebar */
[data-testid="stSidebar"] {
    background-color: var(--secondary-color) !important;
    border-right: 1px solid #444;
    box-shadow: 2px 0 10px rgba(0,0,0,0.3);
    color: white;
}

/* Cards */
.custom-card {
    background: var(--secondary-color);
    color: white;
    border-radius: 10px;
    padding: 1.5rem;
    box-shadow: 0 4px 12px rgba(0,0,0,0.2);
    margin-bottom: 1.5rem;
    border-left: 4px solid var(--primary-color);
}

/* Buttons */
.stButton>button {
    border-radius: 8px !important;
    border: none !important;
    background-color: var(--primary-color) !important;
    color: white !important;
    transition: all 0.3s ease !important;
}

.stButton>button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.3) !important;
}

/* Tabs */
[data-baseweb="tab-list"] {
    gap: 10px;
}

[data-baseweb="tab"] {
    padding: 8px 16px !important;
    border-radius: 8px !important;
    background-color: #34495e !important;
    color: white !important;
    transition: all 0.3s ease !important;
}

[data-baseweb="tab"]:hover {
    background-color: #2c3e50 !important;
}

[aria-selected="true"] {
    background-color: var(--primary-color) !important;
    color: white !important;
}

/* Progress bar */
.stProgress > div > div > div > div {
    background-color: var(--primary-color) !important;
}

/* Tables */
.stDataFrame {
    background-color: var(--secondary-color) !important;
    color: white !important;
}
"""

with open('styles.css', 'w') as f:
    f.write(custom_css)
local_css('styles.css')

# ====== ANIMATION & EFFECTS ======
def render_animated_header():
    st.markdown("""
    <div style="display: flex; align-items: center; justify-content: space-between;">
        <div>
            <h1 style="color: white; margin-bottom: 0;">🌍 AirVision Analytics</h1>
            <p style="color: white; opacity: 0.9; margin-top: 0.5rem;">Trực quan hóa dữ liệu chất lượng không khí & khí tượng</p>
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 0.5rem 1rem; border-radius: 20px; display: flex; align-items: center;">
            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg" style="margin-right: 0.5rem;">
                <path d="M12 2C6.48 2 2 6.48 2 12C2 17.52 6.48 22 12 22C17.52 22 22 17.52 22 12C22 6.48 17.52 2 12 2ZM12 20C7.59 20 4 16.41 4 12C4 7.59 7.59 4 12 4C16.41 4 20 7.59 20 12C20 16.41 16.41 20 12 20Z" fill="white"/>
                <path d="M12 6C8.69 6 6 8.69 6 12C6 15.31 8.69 18 12 18C15.31 18 18 15.31 18 12C18 8.69 15.31 6 12 6ZM12 16C9.79 16 8 14.21 8 12C8 9.79 9.79 8 12 8C14.21 8 16 9.79 16 12C16 14.21 14.21 16 12 16Z" fill="white"/>
                <path d="M12 10C10.9 10 10 10.9 10 12C10 13.1 10.9 14 12 14C13.1 14 14 13.1 14 12C14 10.9 13.1 10 12 10Z" fill="white"/>
            </svg>
            <span style="color: white;">Real-time Monitoring</span>
        </div>
    </div>
    """, unsafe_allow_html=True)

# ====== HÀM TẢI DỮ LIỆU ======
@st.cache_data
def load_data():
    url = "https://redcap.huph.edu.vn/ddp/tsa/nhom4.csv"
    try:
        # Đọc file CSV từ URL
        df = pd.read_csv(url, sep=',', decimal='.', na_values=-200)

        # Chọn các cột cần thiết
        cols = ['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
                'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)',
                'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']
        df = df[cols]

        # Chuyển đổi sang datetime và đặt làm index
        df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        df = df.drop(columns=['Date', 'Time']).set_index('Timestamp')

        # Xử lý giá trị thiếu
        df = df.dropna()

        return df

    except Exception as e:
        st.error(f"Lỗi khi đọc dữ liệu: {str(e)}")
        return None


# ====== MAIN APP ======
def main():
    render_animated_header()

    # Tạo hiệu ứng loading
    with st.spinner('Đang tải dữ liệu và khởi tạo ứng dụng...'):
        time.sleep(1)

    # Tải dữ liệu
    df = load_data()

    if df is None or df.empty:
        st.error("Không thể tải dữ liệu hoặc dữ liệu trống. Vui lòng kiểm tra kết nối hoặc định dạng dữ liệu.")
        return

    # Kiểm tra các cột có trong dữ liệu
    available_columns = df.columns.tolist()
    pollutants = ['CO(GT)', 'NOx(GT)', 'PT08.S5(O3)', 'C6H6(GT)', 'NMHC(GT)', 'NO2(GT)']
    weather = ['T', 'RH', 'AH']

    # ====== SIDEBAR HIỆN ĐẠI ======
    with st.sidebar:
        st.markdown("""
        <div style="display: flex; align-items: center; margin-bottom: 1.5rem;">
            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg" style="margin-right: 0.5rem;">
                <path d="M10 20V14H14V20H19V12H22L12 3L2 12H5V20H10Z" fill="#4a8fe7"/>
            </svg>
            <h2 style="margin: 0; color: white;">Bộ lọc dữ liệu</h2>
        </div>
        """, unsafe_allow_html=True)

        # Date/Time Filter
        with st.expander("🗓️ Lọc theo thời gian", expanded=True):
            min_date = df.index.min().date()
            max_date = df.index.max().date()

            col1, col2 = st.columns(2)
            with col1:
                start_date = st.date_input("Từ ngày", min_date, key="start_date")
            with col2:
                end_date = st.date_input("Đến ngày", max_date, key="end_date")

            col3, col4 = st.columns(2)
            with col3:
                start_time = st.time_input("Từ giờ", datetime.strptime("00:00:00", "%H:%M:%S").time(), key="start_time")
            with col4:
                end_time = st.time_input("Đến giờ", datetime.strptime("23:59:59", "%H:%M:%S").time(), key="end_time")

        # Biến lựa chọn
        with st.expander("📊 Chọn biến hiển thị", expanded=True):
            st.markdown("**Chất ô nhiễm**", help="Chọn các chất ô nhiễm để hiển thị")
            selected_pollutants_flags = [] # Renamed to avoid conflict
            for poll in pollutants:
                if poll in available_columns:
                    selected_pollutants_flags.append(st.checkbox(poll, True, key=f"poll_{poll}"))
                else:
                    st.warning(f"Không tìm thấy cột {poll} trong dữ liệu")

            st.markdown("**Thông số khí tượng**", help="Chọn các thông số khí tượng để hiển thị")
            selected_weather_flags = [] # Renamed to avoid conflict
            for w in weather:
                if w in available_columns:
                    selected_weather_flags.append(st.checkbox(w, True, key=f"weather_{w}"))
                else:
                    st.warning(f"Không tìm thấy cột {w} trong dữ liệu")

    # ====== MAIN DASHBOARD ======
    # Lọc dữ liệu theo thời gian
    try:
        start_dt = datetime.combine(start_date, start_time)
        end_dt = datetime.combine(end_date, end_time)
        df_filtered = df[(df.index >= start_dt) & (df.index <= end_dt)]
    except Exception as e:
        st.error(f"Lỗi khi lọc dữ liệu theo thời gian: {str(e)}")
        df_filtered = df.copy()

    # Tạo tabs
    tab1, tab2, tab3, tab4 = st.tabs(["📈 Dashboard", "🔍 Phân tích", "📤 Dữ liệu", "📊 Kỹ thuật"])

    with tab1:
        # Thống kê nhanh
        st.markdown("### 📊 Tổng quan dữ liệu")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.markdown(f"""
            <div class="custom-card">
                <h3 style="color: var(--primary-color); margin-top: 0;">Thời gian</h3>
                <p>Bắt đầu: {df_filtered.index.min().strftime('%d/%m/%Y %H:%M')}</p>
                <p>Kết thúc: {df_filtered.index.max().strftime('%d/%m/%Y %H:%M')}</p>
            </div>
            """, unsafe_allow_html=True)

        with col2:
            st.markdown(f"""
            <div class="custom-card">
                <h3 style="color: var(--primary-color); margin-top: 0;">Số lượng dữ liệu</h3>
                <p>Tổng số mẫu: {len(df_filtered):,}</p>
                <p>Số ngày: {(df_filtered.index.max() - df_filtered.index.min()).days + 1}</p>
            </div>
            """, unsafe_allow_html=True)

        with col3:
            avg_co = df_filtered['CO(GT)'].mean() if 'CO(GT)' in df_filtered.columns else 'N/A'
            avg_nox = df_filtered['NOx(GT)'].mean() if 'NOx(GT)' in df_filtered.columns else 'N/A'
            st.markdown(f"""
            <div class="custom-card">
                <h3 style="color: var(--primary-color); margin-top: 0;">Chất lượng không khí</h3>
                <p>CO trung bình: {avg_co if isinstance(avg_co, str) else f"{avg_co:.2f} ppm"}</p>
                <p>NOx trung bình: {avg_nox if isinstance(avg_nox, str) else f"{avg_nox:.2f} ppb"}</p>
            </div>
            """, unsafe_allow_html=True)

        st.markdown("### ⚠️ Cảnh báo ngưỡng ô nhiễm")

        # Định nghĩa ngưỡng cảnh báo (có thể thay đổi theo tiêu chuẩn)
        thresholds = {
            'CO(GT)': {'low': 0, 'medium': 5, 'high': 10},  # ppm
            'NOx(GT)': {'low': 0, 'medium': 50, 'high': 100},  # ppb
            'C6H6(GT)': {'low': 0, 'medium': 5, 'high': 10},  # µg/m³
            'PT08.S5(O3)': {'low': 0, 'medium': 100, 'high': 200},  # ppb
            'NMHC(GT)': {'low': 0, 'medium': 200, 'high': 400},  # ppm
            'NO2(GT)': {'low': 0, 'medium': 100, 'high': 200}  # ppb
        }

        # Tính toán số lần vượt ngưỡng
        alerts = []
        for poll, levels in thresholds.items():
            if poll in df_filtered.columns:
                high_alerts = len(df_filtered[df_filtered[poll] > levels['high']])
                med_alerts = len(df_filtered[(df_filtered[poll] > levels['medium']) & (df_filtered[poll] <= levels['high'])])

                alerts.append({
                    'Chất ô nhiễm': poll,
                    'Vượt ngưỡng cao': high_alerts,
                    'Vượt ngưỡng trung bình': med_alerts,
                    'Ngưỡng cao': levels['high'],
                    'Ngưỡng trung bình': levels['medium']
                })

        if alerts:
            # Hiển thị bảng cảnh báo
            st.dataframe(
                pd.DataFrame(alerts),
                use_container_width=True,
                hide_index=True
            )

            # Hiển thị biểu đồ cảnh báo
            fig_alerts = go.Figure() # Renamed to avoid conflict
            for poll_alert_data in alerts: # Iterate through alert data to get correct poll name
                poll_name = poll_alert_data['Chất ô nhiễm']
                if poll_name in df_filtered.columns:
                    fig_alerts.add_trace(go.Scatter(
                        x=df_filtered.index,
                        y=df_filtered[poll_name],
                        name=poll_name,
                        mode='lines'
                    ))

                    # Thêm đường ngưỡng
                    fig_alerts.add_hline(
                        y=thresholds[poll_name]['high'],
                        line_dash="dot",
                        line_color="red",
                        annotation_text=f"Ngưỡng cao {poll_name}",
                        annotation_position="bottom right"
                    )
                    fig_alerts.add_hline(
                        y=thresholds[poll_name]['medium'],
                        line_dash="dash",
                        line_color="orange",
                        annotation_text=f"Ngưỡng TB {poll_name}",
                        annotation_position="bottom right"
                    )

            fig_alerts.update_layout(
                title="Cảnh báo vượt ngưỡng ô nhiễm",
                yaxis_title="Nồng độ",
                template='plotly_dark'
            )
            st.plotly_chart(fig_alerts, use_container_width=True)
        else:
            st.info("Không có dữ liệu chất ô nhiễm để kiểm tra ngưỡng")

        # Biểu đồ chính
        st.markdown("### 🌡️ Biểu đồ theo dõi")
        if not df_filtered.empty:
            fig_main_dashboard = make_subplots(specs=[[{"secondary_y": True}]]) # Renamed to avoid conflict

            # Thêm dữ liệu chất ô nhiễm
            for poll in [p for p in pollutants if p in df_filtered.columns]:
                fig_main_dashboard.add_trace(
                    go.Scatter(
                        x=df_filtered.index,
                        y=df_filtered[poll],
                        name=poll,
                        line=dict(width=2),
                        mode='lines',
                        hovertemplate="%{y:.2f}<extra></extra>"
                    ),
                    secondary_y=False
                )

            # Thêm dữ liệu khí tượng
            for w in [w_param for w_param in weather if w_param in df_filtered.columns]: # Renamed inner variable
                fig_main_dashboard.add_trace(
                    go.Scatter(
                        x=df_filtered.index,
                        y=df_filtered[w],
                        name=w,
                        line=dict(dash='dot', width=1.5),
                        mode='lines',
                        hovertemplate="%{y:.2f}<extra></extra>"
                    ),
                    secondary_y=True
                )

            fig_main_dashboard.update_layout(
                height=500,
                template='plotly_dark',
                hovermode="x unified",
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                margin=dict(l=20, r=20, t=40, b=20),
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )

            fig_main_dashboard.update_yaxes(title_text="Nồng độ chất ô nhiễm", secondary_y=False)
            fig_main_dashboard.update_yaxes(title_text="Thông số khí tượng", secondary_y=True)

            st.plotly_chart(fig_main_dashboard, use_container_width=True)

    with tab2:
        st.markdown("### 🔥 Phân tích tương quan")
        # Lọc các cột có trong dữ liệu
        available_vars = [v for v in pollutants + weather if v in df_filtered.columns]

        if len(available_vars) >= 2:
            corr_matrix = df_filtered[available_vars].corr()

            fig_corr, ax_corr = plt.subplots(figsize=(10, 8)) # Renamed
            sns.heatmap(
                corr_matrix,
                annot=True,
                fmt=".2f",
                cmap="coolwarm",
                center=0,
                square=True,
                linewidths=.5,
                cbar_kws={"shrink": .8},
                ax=ax_corr # Use renamed ax
            )
            plt.title("Ma trận tương quan giữa các thông số", pad=20, color='white')
            ax_corr.set_facecolor('#2c3e50') # Use renamed ax
            fig_corr.patch.set_facecolor('#2c3e50') # Use renamed fig
            ax_corr.tick_params(axis='x', colors='white') # Use renamed ax
            ax_corr.tick_params(axis='y', colors='white') # Use renamed ax
            st.pyplot(fig_corr) # Use renamed fig

            # Phân tích xu hướng
            st.markdown("### 📉 Phân tích xu hướng")
            selected_trend = st.selectbox("Chọn thông số để phân tích xu hướng", available_vars)

            if selected_trend in df_filtered.columns:
                trend_fig = go.Figure()

                trend_fig.add_trace(
                    go.Scatter(
                        x=df_filtered.index,
                        y=df_filtered[selected_trend],
                        name='Giá trị thực',
                        line=dict(color='#4a8fe7')
                    )
                )

                # Thêm đường trung bình 7 ngày
                rolling_mean = df_filtered[selected_trend].rolling('7D').mean()
                trend_fig.add_trace(
                    go.Scatter(
                        x=rolling_mean.index,
                        y=rolling_mean.values,
                        name='Trung bình 7 ngày',
                        line=dict(color='#ff6b6b', dash='dash')
                    )
                )

                trend_fig.update_layout(
                    title=f"Xu hướng {selected_trend}",
                    xaxis_title="Thời gian",
                    yaxis_title="Giá trị",
                    template='plotly_dark',
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)'
                )

                st.plotly_chart(trend_fig, use_container_width=True)

    with tab3:
        st.markdown("### 📂 Dữ liệu thô")

        # Hiển thị toàn bộ dữ liệu dạng bảng
        st.dataframe(
            df_filtered,
            height=600,  # Chiều cao cố định
            use_container_width=True,
            hide_index=True  # Ẩn chỉ số dòng
        )

        # Thông tin tóm tắt trong expander
        with st.expander("ℹ️ Thông tin dữ liệu"):
            cols_info = st.columns(3) # Renamed to avoid conflict
            with cols_info[0]:
                st.metric("Số dòng", len(df_filtered))
            with cols_info[1]:
                st.metric("Số cột", len(df_filtered.columns))
            with cols_info[2]:
                st.metric("Dữ liệu thiếu", df_filtered.isna().sum().sum())

        # Tùy chọn xuất dữ liệu
        st.markdown("### 📤 Xuất dữ liệu")
        export_format = st.radio("Định dạng xuất", ["CSV", "Excel"], horizontal=True)

        if st.button("Xuất dữ liệu đã lọc", type="primary"):
            if export_format == "CSV":
                csv_data = df_filtered.to_csv(index=True).encode('utf-8') # Renamed
                st.download_button(
                    label="Tải xuống CSV",
                    data=csv_data, # Use renamed
                    file_name=f"air_quality_{start_date}_to_{end_date}.csv",
                    mime="text/csv"
                )
            else:
                from io import BytesIO
                output = BytesIO()
                with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
                    df_filtered.to_excel(writer, index=True)
                excel_data = output.getvalue()
                st.download_button(
                    label="Tải xuống Excel",
                    data=excel_data,
                    file_name=f"air_quality_{start_date}_to_{end_date}.xlsx",
                    mime="application/vnd.ms-excel"
                )
    with tab4:
        st.markdown("### 📊 Phân tích chuỗi thời gian nâng cao")

        # Chọn biến để phân tích
        tech_analysis_var = st.selectbox(
            "Chọn biến để phân tích kỹ thuật",
            [col for col in df.columns if col not in ['Date', 'Time']], # Ensure df is used, not df_filtered for full list
            key='tech_var'
        )

        if tech_analysis_var and tech_analysis_var in df_filtered.columns: # Check if var exists in filtered data
            series = df_filtered[tech_analysis_var].dropna()

            if series.empty:
                st.warning(f"Không có dữ liệu cho '{tech_analysis_var}' trong khoảng thời gian đã chọn sau khi loại bỏ NA.")
            else:
                # Tạo các subtabs trong tab kỹ thuật
                subtab1, subtab2, subtab3, subtab4 = st.tabs([
                    "Thành phần mùa vụ",
                    "Tính dừng",
                    "Tự tương quan",
                    "Xu hướng & Ngoại lệ"
                ])

                with subtab1:
                    st.markdown(f"#### Phân tích thành phần mùa vụ - {tech_analysis_var}")
                    if len(series) >= 48: # Period is 24, so need at least 2*period
                        try:
                            # Tạo thư mục lưu ảnh nếu chưa tồn tại
                            if not os.path.exists("charts/decomposition"):
                                os.makedirs("charts/decomposition")

                            # Thực hiện decomposition
                            decomposition = seasonal_decompose(series, model='additive', period=24)

                            # Vẽ biểu đồ decomposition using Matplotlib
                            fig_decomp = decomposition.plot() # Renamed
                            fig_decomp.suptitle(f"Decomposition of {tech_analysis_var}", y=1.02)

                            # Tùy chỉnh màu sắc
                            axes_decomp = fig_decomp.get_axes() # Renamed
                            if len(axes_decomp) > 1 and len(axes_decomp[1].lines) > 0 : axes_decomp[1].lines[0].set_color('red')   # Trend
                            if len(axes_decomp) > 2 and len(axes_decomp[2].lines) > 0 : axes_decomp[2].lines[0].set_color('green') # Seasonal
                            if len(axes_decomp) > 3 and len(axes_decomp[3].lines) > 0 : axes_decomp[3].lines[0].set_color('orange')# Residual

                            # Thêm grid
                            for ax_d in axes_decomp: # Renamed inner var
                                ax_d.grid(True)

                            plt.tight_layout()

                            # Lưu file ảnh tạm
                            temp_file_path = f"charts/decomposition/{tech_analysis_var}_decomp.png" # Renamed
                            plt.savefig(temp_file_path, bbox_inches='tight')
                            plt.close(fig_decomp) # Close the figure

                            # Hiển thị ảnh trong Streamlit
                            with open(temp_file_path, "rb") as img_file: # Renamed
                                img_data = base64.b64encode(img_file.read()).decode()
                                img_html = f'<img src="data:image/png;base64,{img_data}" style="width:100%">'
                                st.markdown(img_html, unsafe_allow_html=True)

                            # Giải thích kết quả
                            st.markdown("""
                            **Giải thích thành phần:**
                            - **Observed**: Dữ liệu gốc
                            - **Trend**: Xu hướng dài hạn
                            - **Seasonal**: Biến động theo mùa (chu kỳ)
                            - **Residual**: Phần dư (nhiễu ngẫu nhiên)
                            """)

                        except Exception as e:
                            st.error(f"Không thể phân tích thành phần mùa vụ: {str(e)}")
                            st.error(f"Chi tiết lỗi: {repr(e)}")
                    else:
                        st.warning(f"Không đủ dữ liệu để phân tích thành phần mùa vụ (cần ít nhất 48 quan sát, hiện có {len(series)})")

                with subtab2:
                    st.markdown(f"#### Kiểm tra tính dừng - {tech_analysis_var}")
                    if not series.empty:
                        try:
                            result_adfuller = adfuller(series) # Renamed

                            col_adf1, col_adf2 = st.columns(2) # Renamed
                            with col_adf1:
                                st.metric("ADF Statistic", f"{result_adfuller[0]:.4f}")
                                st.metric("p-value", f"{result_adfuller[1]:.4f}")

                            with col_adf2:
                                st.write("**Critical Values:**")
                                for key, value in result_adfuller[4].items():
                                    st.write(f"{key}: {value:.4f}")

                            if result_adfuller[1] > 0.05:
                                st.warning("Chuỗi KHÔNG dừng (non-stationary)")
                                st.markdown("**Gợi ý:** Cần áp dụng phương pháp như differencing để làm chuỗi dừng")
                            else:
                                st.success("Chuỗi dừng (stationary)")

                            # Hiển thị biểu đồ so sánh với rolling mean/std
                            fig_stationarity = go.Figure() # Renamed
                            fig_stationarity.add_trace(go.Scatter(x=series.index, y=series, name='Original'))
                            fig_stationarity.add_trace(go.Scatter(x=series.index, y=series.rolling(window=12).mean(), name='Rolling Mean'))
                            fig_stationarity.add_trace(go.Scatter(x=series.index, y=series.rolling(window=12).std(), name='Rolling Std'))

                            fig_stationarity.update_layout(
                                title=f"Kiểm tra tính dừng - {tech_analysis_var}",
                                template='plotly_dark'
                            )
                            st.plotly_chart(fig_stationarity, use_container_width=True)

                        except Exception as e:
                            st.error(f"Không thể thực hiện ADF test: {str(e)}")
                    else:
                        st.warning(f"Không có dữ liệu cho '{tech_analysis_var}' để kiểm tra tính dừng.")


                with subtab3:
                    st.markdown(f"#### Phân tích tự tương quan - {tech_analysis_var}")
                    if not series.empty:
                        col_acf, col_pacf = st.columns(2) # Renamed
                        with col_acf:
                            st.markdown("**Autocorrelation (ACF)**")
                            fig_acf = plot_acf(series, lags=40) # Renamed
                            st.pyplot(fig_acf)
                            plt.close(fig_acf) # Close the figure

                        with col_pacf:
                            st.markdown("**Partial Autocorrelation (PACF)**")
                            fig_pacf = plot_pacf(series, lags=40, method='ywmle') # Renamed, ywmle can cause issues with small samples
                            st.pyplot(fig_pacf)
                            plt.close(fig_pacf) # Close the figure


                        st.markdown("""
                        **Giải thích:**
                        - ACF giúp xác định bậc q trong MA(q)
                        - PACF giúp xác định bậc p trong AR(p)
                        """)
                    else:
                        st.warning(f"Không có dữ liệu cho '{tech_analysis_var}' để phân tích tự tương quan.")

                with subtab4:
                    st.markdown(f"#### Phát hiện ngoại lệ - {tech_analysis_var}")
                    if not series.empty:
                        # Tính toán và hiển thị outliers sử dụng IQR
                        q1 = series.quantile(0.25)
                        q3 = series.quantile(0.75)
                        iqr = q3 - q1
                        lower_bound = q1 - 1.5 * iqr
                        upper_bound = q3 + 1.5 * iqr

                        outliers = series[(series < lower_bound) | (series > upper_bound)]

                        col_outlier1, col_outlier2 = st.columns(2) # Renamed
                        with col_outlier1:
                            st.metric("Số lượng ngoại lệ", len(outliers))
                            if len(series) > 0 :
                                st.metric("Tỷ lệ ngoại lệ", f"{len(outliers)/len(series):.2%}")
                            else:
                                st.metric("Tỷ lệ ngoại lệ", "N/A")


                        with col_outlier2:
                            st.metric("Ngưỡng dưới", f"{lower_bound:.2f}")
                            st.metric("Ngưỡng trên", f"{upper_bound:.2f}")

                        # Hiển thị biểu đồ với outliers
                        fig_outliers = go.Figure() # Renamed
                        fig_outliers.add_trace(go.Scatter(
                            x=series.index,
                            y=series,
                            mode='lines',
                            name='Dữ liệu'
                        ))
                        if not outliers.empty:
                            fig_outliers.add_trace(go.Scatter(
                                x=outliers.index,
                                y=outliers,
                                mode='markers',
                                marker=dict(color='red', size=8),
                                name='Ngoại lệ'
                            ))

                        fig_outliers.update_layout(
                            title=f"Phát hiện ngoại lệ - {tech_analysis_var}",
                            template='plotly_dark'
                        )
                        st.plotly_chart(fig_outliers, use_container_width=True)

                        # Xử lý outliers
                        st.markdown("**Xử lý ngoại lệ**")
                        method_outlier = st.selectbox( # Renamed
                            "Phương pháp xử lý",
                            ["Không xử lý", "Cắt ngưỡng", "Thay thế bằng trung vị", "Xóa bỏ"],
                            key='outlier_method'
                        )

                        if st.button("Áp dụng xử lý"):
                            series_processed = series.copy() # Start with a copy
                            if method_outlier == "Cắt ngưỡng":
                                series_processed = series.clip(lower_bound, upper_bound)
                            elif method_outlier == "Thay thế bằng trung vị":
                                median_val = series.median() # Renamed
                                series_processed = series.where(~((series < lower_bound) | (series > upper_bound)), median_val)
                            elif method_outlier == "Xóa bỏ":
                                series_processed = series[~series.index.isin(outliers.index)]

                            # Hiển thị kết quả
                            fig_processed = go.Figure() # Renamed
                            fig_processed.add_trace(go.Scatter(
                                x=series_processed.index,
                                y=series_processed,
                                mode='lines',
                                name='Dữ liệu đã xử lý'
                            ))

                            fig_processed.update_layout(
                                title=f"Dữ liệu sau xử lý - {tech_analysis_var}",
                                template='plotly_dark'
                            )
                            st.plotly_chart(fig_processed, use_container_width=True)

                            # Cập nhật dataframe (Important: This modifies df_filtered for the current session only)
                            # df_filtered[tech_analysis_var] = series_processed # Be cautious with direct modification
                            st.success("Đã áp dụng xử lý ngoại lệ (hiển thị). Dữ liệu gốc không thay đổi cho các lần chạy sau trừ khi lưu lại.")
                    else:
                        st.warning(f"Không có dữ liệu cho '{tech_analysis_var}' để phát hiện ngoại lệ.")


            st.markdown("### 🔮 Đánh giá mô hình dự báo đơn giản")

            if tech_analysis_var and tech_analysis_var in df_filtered.columns:
                series_for_forecast = df_filtered[tech_analysis_var].dropna() # Use the potentially processed series or original filtered
                if len(series_for_forecast) > 10: # Need enough data for train/test
                    # Chia train/test
                    train_size = int(len(series_for_forecast) * 0.8)
                    train, test = series_for_forecast[:train_size], series_for_forecast[train_size:]

                    if not test.empty and len(test) > 1:
                        # Mô hình naive (sử dụng giá trị trước đó)
                        naive_forecast = test.shift(1).dropna() # Drop first NaN from shift
                        actual_values = test[1:] # Align with naive_forecast

                        if not naive_forecast.empty and not actual_values.empty and len(naive_forecast) == len(actual_values):
                            # Tính toán các chỉ số
                            mse = mean_squared_error(actual_values, naive_forecast)
                            rmse = np.sqrt(mse)
                            mae = np.mean(np.abs(actual_values - naive_forecast))
                            mean_actual = np.mean(actual_values) # Renamed

                            col_metric1, col_metric2, col_metric3 = st.columns(3) # Renamed
                            with col_metric1:
                                st.metric("RMSE", f"{rmse:.2f}")
                            with col_metric2:
                                st.metric("MAE", f"{mae:.2f}")
                            with col_metric3:
                                if mean_actual != 0:
                                    st.metric("Tỷ lệ lỗi (MAE/Mean)", f"{mae/mean_actual:.2%}")
                                else:
                                    st.metric("Tỷ lệ lỗi (MAE/Mean)", "N/A (Mean is 0)")


                            # Hiển thị kết quả dự báo
                            fig_forecast = go.Figure() # Renamed
                            fig_forecast.add_trace(go.Scatter(
                                x=train.index,
                                y=train,
                                name='Train'
                            ))
                            fig_forecast.add_trace(go.Scatter(
                                x=test.index,
                                y=test,
                                name='Test'
                            ))
                            fig_forecast.add_trace(go.Scatter(
                                x=naive_forecast.index,
                                y=naive_forecast,
                                name='Dự báo Naive',
                                line=dict(dash='dot')
                            ))

                            fig_forecast.update_layout(
                                title=f"So sánh dự báo Naive - {tech_analysis_var}",
                                template='plotly_dark'
                            )
                            st.plotly_chart(fig_forecast, use_container_width=True)

                            st.markdown("""
                            **Giải thích:**
                            - Mô hình Naive sử dụng giá trị tại thời điểm t-1 để dự báo cho thời điểm t.
                            - Các chỉ số càng thấp càng tốt.
                            - Đây là baseline model để so sánh với các mô hình phức tạp hơn.
                            """)
                        else:
                            st.warning("Không đủ dữ liệu sau khi tạo dự báo naive để đánh giá.")
                    else:
                        st.warning("Không đủ dữ liệu trong tập test để thực hiện dự báo naive.")
                else:
                    st.warning(f"Không đủ dữ liệu cho '{tech_analysis_var}' để thực hiện đánh giá mô hình dự báo (cần > 10 điểm dữ liệu).")
            else:
                st.info("Chọn một biến từ menu trên để xem đánh giá mô hình dự báo.")


if __name__ == "__main__":
    main()
'''

with open('/content/app.py', 'w') as f:
    f.write(app_code)

In [20]:
# --- 3. Khởi chạy ứng dụng ---
from pyngrok import ngrok
import subprocess
import threading
import time
import requests

# Reset ngrok tunnels
ngrok.kill()

# Cấu hình Ngrok
NGROK_TOKEN = "2wIdHLEeJI9sBLsjQ3Lkt30ZYkT_33pMAYJokqqqyypmD9Zz9"  # 👈 Thay token của bạn
ngrok.set_auth_token(NGROK_TOKEN)

# Khởi chạy Streamlit
def run_streamlit():
    subprocess.run([
        "streamlit", "run",
        "/content/app.py",
        "--server.port", "8501",
        "--server.headless", "true",
        "--browser.gatherUsageStats", "false"
    ])

threading.Thread(target=run_streamlit, daemon=True).start()

# Chờ khởi động
time.sleep(8)

# Tạo public URL
try:
    tunnel = ngrok.connect(8501, "http")
    public_url = tunnel.public_url
    print(f"\n🔗 Truy cập ứng dụng tại: {public_url}")
    print(f"📊 Xem dashboard Ngrok: https://dashboard.ngrok.com/status/tunnels")

except Exception as e:
    print(f"\n❌ Lỗi kết nối: {str(e)}")
    print("👉 Nguyên nhân thường gặp:")
    print("- Hết hạn token Ngrok")
    print("- Chạy quá 3 tunnel cùng lúc (tài khoản miễn phí)")
    print("- Lỗi mạng")



🔗 Truy cập ứng dụng tại: https://6bd7-34-147-113-9.ngrok-free.app
📊 Xem dashboard Ngrok: https://dashboard.ngrok.com/status/tunnels
